<a href="https://colab.research.google.com/github/AdryGP/ABP-FindMo/blob/main/FindMo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FindMo: Manual de Uso

---



# Paso 1: Carga de Datasets Originales

Primero, cargamos los datasets originales, almacenados en un drive público:

In [ ]:
# !pip install gdown

!mkdir -p original_datasets/videogames 

!mkdir -p original_datasets/movies 

!mkdir -p system_datasets

import gdown as gd

# Videojuegos
gd.download('https://drive.google.com/uc?id=1-KgMce1i1IuEEsk-xUnIRTWq1R8jr8Ey', 'original_datasets/videogames/steam_list.csv', quiet=False) # Títulos de videojuegos (Steam)

gd.download('https://drive.google.com/uc?id=1uYAG398bhK7fylytYixJo8xdHirFd_Xe', 'original_datasets/videogames/steam_descriptions.csv', quiet=False) # Descripciones de videojuegos (Steam)

gd.download('https://drive.google.com/uc?id=1IqKNR6cdWJWIDXZSxg-TnRITpFWv2tgM', 'original_datasets/videogames/steam_reviews.csv', quiet=False) # Comentarios de videojuegos. (Steam)

# Películas
gd.download('https://drive.google.com/uc?id=1U7OJMy5wmseeeRLiPhwnXRsHzB0Az3-I', 'original_datasets/movies/imbd_movies.csv', quiet=False) # Títulos y descripciones de películas (IMDb)

gd.download('https://drive.google.com/uc?id=151yjtQnjfC1PxUiyw1xVXhkmHeBVwL0U', 'original_datasets/movies/metacritic_movies.csv', quiet=False) # Títulos de películas (Metacritic)

gd.download('https://drive.google.com/uc?id=157m8NSM5D7xE-VZ8iqVzNNvQa6rMPf3d', 'original_datasets/movies/metacritic_reviews.csv', quiet=False) # Comentarios de películas (Metacritic)

Downloading...
From: https://drive.google.com/uc?id=1-KgMce1i1IuEEsk-xUnIRTWq1R8jr8Ey
To: /content/original_datasets/videogames/steam_list.csv
5.82MB [00:00, 65.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uYAG398bhK7fylytYixJo8xdHirFd_Xe
To: /content/original_datasets/videogames/steam_descriptions.csv
94.7MB [00:00, 204MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IqKNR6cdWJWIDXZSxg-TnRITpFWv2tgM
To: /content/original_datasets/videogames/steam_reviews.csv
121MB [00:00, 178MB/s]
Downloading...
From: https://drive.google.com/uc?id=1U7OJMy5wmseeeRLiPhwnXRsHzB0Az3-I
To: /content/original_datasets/movies/imbd_movies.csv
47.9MB [00:00, 198MB/s] 
Downloading...
From: https://drive.google.com/uc?id=151yjtQnjfC1PxUiyw1xVXhkmHeBVwL0U
To: /content/original_datasets/movies/metacritic_movies.csv
100%|██████████| 636k/636k [00:00<00:00, 94.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=157m8NSM5D7xE-VZ8iqVzNNvQa6rMPf3d
To: /content/original_datasets/movies/me

'original_datasets/movies/metacritic_reviews.csv'

# Paso 2: Confección de Datasets Propios seleccionando un subconjuntos de los Datasets Originales

Abrimos los ficheros con Pandas:

In [ ]:
import pandas as pd

# Videojuegos
originalSteamList = pd.read_csv('original_datasets/videogames/steam_list.csv')
originalSteamDescriptions = pd.read_csv('original_datasets/videogames/steam_descriptions.csv')
originalSteamReviews = pd.read_csv('original_datasets/videogames/steam_reviews.csv')

# Películas
originalIMBDMoviesWithDescription = pd.read_csv('original_datasets/movies/imbd_movies.csv')
originalMetacriticMovies = pd.read_csv('original_datasets/movies/metacritic_movies.csv')
originalMetacriticReviewa = pd.read_csv('original_datasets/movies/metacritic_reviews.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Reducimos el tamaño del dataset de videojuegos, solo considerando aquellos juegos que han vendido más de 20k unidades

In [ ]:
originalSteamList = originalSteamList[originalSteamList['owners'] != '0-20000']

Reducimos el tamaño del dataset de cine, solo considerando películas con más de 70 reseñas por usuarios

In [ ]:
originalIMBDMoviesWithDescription = originalIMBDMoviesWithDescription[originalIMBDMoviesWithDescription['reviews_from_users'] > 70]

Ahora, unimos datasets y seleccionamos las columnas necesarias:

In [ ]:
# Estas líneas crean una copia del csv con un id para cada elemento con el formato que hemos especificado (<inicialMedio><autoIncremental>):
# Videojuegos
listadoVideojuegos = originalSteamList.copy()
listadoVideojuegos.insert(0, 'id_obra', range(0, len(listadoVideojuegos)))
listadoVideojuegos = listadoVideojuegos.assign(id_obra=lambda x: 'V'+x.id_obra.astype(str))

# Películas
listadoCineDescripcion = originalIMBDMoviesWithDescription.copy()
listadoCineDescripcion.insert(0, 'id_obra', range(0, len(listadoCineDescripcion)))
listadoCineDescripcion = listadoCineDescripcion.assign(id_obra=lambda x: 'C'+x.id_obra.astype(str))


# Creamos el listado que contiene títulos y descripciones (y guardamos el csv):
# Videojuegos
listadoDescripcionVideojuegos = pd.merge(listadoVideojuegos, originalSteamDescriptions, left_on='appid', right_on='steam_appid')

listadoDescripcionVideojuegos = listadoDescripcionVideojuegos.rename(columns={'name':'nombre', 'short_description':'descripcion'})

listadoDescripcionVideojuegos = listadoDescripcionVideojuegos.assign(medio='V')

# Almacenamos el listado obtenido en un fichero CSV
listadoDescripcionVideojuegos[['id_obra', 'medio', 'nombre', 'descripcion']].to_csv('system_datasets/obras_descripcion.csv', index=False)

listadoDescripcionVideojuegos[['id_obra', 'medio', 'nombre', 'descripcion']]

,id_obra,medio,nombre,descripcion
0,V0,V,Counter-Strike,Play the world's number 1 online action game. ...
1,V1,V,Team Fortress Classic,One of the most popular online action games of...
2,V2,V,Day of Defeat,Enlist in an intense brand of Axis vs. Allied ...
3,V3,V,Deathmatch Classic,Enjoy fast-paced multiplayer gaming with Death...
4,V4,V,Half-Life: Opposing Force,Return to the Black Mesa Research Facility as ...
...,...,...,...,...
8474,V8474,V,BLOCKADE Classic,BLOCKADE Classic! First Person Shooter in an e...
8475,V8475,V,Arena Stars,"Choose your Hero, build your Team and take on ..."
8476,V8476,V,Burning Daylight,"Waking up naked in a slaughterhouse, you must ..."
8477,V8477,V,KillStreak.tv,KILLSTREAK.tv is a quick fire 3rd-person multi...


In [ ]:
# Cine:
listadoCineDescripcion = listadoCineDescripcion.rename(columns={'title':'nombre', 'description':'descripcion'})
listadoCineDescripcion = listadoCineDescripcion.assign(medio='C')
listadoCineDescripcion[['id_obra', 'medio', 'nombre', 'descripcion']].dropna().to_csv('system_datasets/obras_descripcion.csv', index=False, mode='a', header=False)
listadoCineDescripcion[['id_obra', 'medio', 'nombre', 'descripcion']].dropna()


,id_obra,medio,nombre,descripcion
47,C0,C,Nascita di una nazione,The Stoneman family finds its friendship with ...
76,C1,C,Intolerance,"The story of a poor young woman, separated by ..."
147,C2,C,Giglio infranto,"A frail waif, abused by her brutal boxer fathe..."
165,C3,C,Il gabinetto del dottor Caligari,"Hypnotist Dr. Caligari uses a somnambulist, Ce..."
190,C4,C,Dr. Jekyll e Mr. Hyde,Dr. Henry Jekyll experiments with scientific m...
...,...,...,...,...
85811,C9130,C,Falling Inn Love - Ristrutturazione con amore,When city girl Gabriela spontaneously enters a...
85817,C9131,C,Paranormal Investigation,When a young man becomes possessed after playi...
85837,C9132,C,VFW,A group of old war veterans put their lives on...
85839,C9133,C,Coffee & Kareem,Twelve-year-old Kareem Manning hires a crimina...


In [ ]:
# Creamos el listado que contiene los comentarios asociados a cada obra:
listadoComentariosVideojuegos = pd.merge(listadoDescripcionVideojuegos, originalSteamReviews, left_on='nombre', right_on='title')

listadoComentariosVideojuegos = listadoComentariosVideojuegos.rename(columns={'review':'comentario', 'recommendation':'valoracion'})

# Almacenamos el listado obtenido en un fichero CSV (se concatena al final del mismo)
listadoComentariosVideojuegos[['id_obra', 'comentario', 'valoracion']].to_csv('system_datasets/obras_comentarios.csv', index=False)

listadoComentariosVideojuegos[['id_obra', 'comentario', 'valoracion']]

,id_obra,comentario,valoracion
0,V21,I used to play this game a lot back when it ca...,Recommended
1,V21,Product received for free. 'No hope. No cure. ...,Recommended
2,V21,cool,Recommended
3,V21,"Well ya see ""TAAAAAAAAAAAAAANK""",Recommended
4,V21,Nice game it's a pity that Gaben doesn't creat...,Recommended
...,...,...,...
434874,V8300,Well having played for a bit now I am pleased ...,Recommended
434875,V8300,runs well without any problem / bug / crash / ...,Recommended
434876,V8300,NO DRM FREE DLC JIGGLE TIDDIES,Recommended
434877,V8300,Pros 2 New Weapon Types introduced with newer ...,Recommended


# Paso 3: Subsistema de Recomendación

## Paso 3.1: Preprocesado del Dataset Propio de obras con descripciones

### Paso 3.1.1: **Tokenizado**, eliminación de **Stopwords** y **Stemming**:

In [ ]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()

preprocessedText = []

originalData = pd.read_csv('system_datasets/obras_descripcion.csv')

for row in originalData.itertuples():
    
    
    text = word_tokenize(row[4]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalpha()]
    text = " ".join(text)
    
    preprocessedText.append(text)

preprocessedData = originalData
preprocessedData['processed_text'] = preprocessedText

preprocessedData

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id_obra,medio,nombre,descripcion,processed_text
0,V0,V,Counter-Strike,Play the world's number 1 online action game. ...,play world number onlin action game engag incr...
1,V1,V,Team Fortress Classic,One of the most popular online action games of...,one popular onlin action game time team fortre...
2,V2,V,Day of Defeat,Enlist in an intense brand of Axis vs. Allied ...,enlist intens brand axi alli teamplay set wwii...
3,V3,V,Deathmatch Classic,Enjoy fast-paced multiplayer gaming with Death...,enjoy multiplay game deathmatch classic dmc va...
4,V4,V,Half-Life: Opposing Force,Return to the Black Mesa Research Facility as ...,return black mesa research facil one militari ...
...,...,...,...,...,...
17592,C9130,C,Falling Inn Love - Ristrutturazione con amore,When city girl Gabriela spontaneously enters a...,when citi girl gabriela spontan enter contest ...
17593,C9131,C,Paranormal Investigation,When a young man becomes possessed after playi...,when young man becom possess play ouija game g...
17594,C9132,C,VFW,A group of old war veterans put their lives on...,A group old war veteran put live line defend y...
17595,C9133,C,Coffee & Kareem,Twelve-year-old Kareem Manning hires a crimina...,kareem man hire crimin scare mom new boyfriend...


### Paso 3.1.2: Creación de la **Bolsa de Palabras**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(preprocessedData['processed_text'])
textsBoW= bagOfWordsModel.transform(preprocessedData['processed_text'])
print("Finished")

Finished


In [ ]:
textsBoW.shape

(17597, 19569)

## Paso 3.2: Cálculo de la **Matriz de Distancias**:

In [ ]:
from sklearn.metrics import pairwise_distances

distance_matrix= pairwise_distances(textsBoW,textsBoW ,metric='cosine')

In [ ]:
print(distance_matrix.shape)

(17597, 17597)
<class 'numpy.ndarray'>


## Paso 3.3: Ejemplo de uso 

In [ ]:
searchTitle = "Portal" # Obra base para las recomendaciones (Otro ejemplo: "The Elder Scrolls V: Skyrim")
indexOfTitle = preprocessedData[preprocessedData['nombre']==searchTitle].index.values[0]
indexOfTitle

23

In [ ]:
distance_scores = list(enumerate(distance_matrix[indexOfTitle]))

In [ ]:
ordered_scores = sorted(distance_scores, key=lambda x: x[1])

In [ ]:
top_scores = ordered_scores[0:10]
#top_scores

In [ ]:
top_indexes = [i[0] for i in top_scores]
#top_indexes

In [ ]:
top = preprocessedData[['id_obra', 'medio', 'nombre', 'descripcion']].iloc[top_indexes]
top = top[top['medio'] != 'V'] # Recomendaciones que no sean videojuegos 
top.head(10)

,id_obra,medio,nombre,descripcion
14789,C6321,C,90 minuti a New York,A perpetually angry man is informed he has 90 ...
15509,C7043,C,Le Mans '66 - La grande sfida,American car designer
12209,C3737,C,The Ladies Man,The
17088,C8623,C,Action Point,A daredevil designs and operates his own theme...
12328,C3856,C,Dark Blue World,The friendship of two men becomes tested when ...
10178,C1702,C,One Dark Night,As part of an initiation into a club called th...
12152,C3680,C,Boys & girls - Attenzione: il sesso cambia tutto,A friendship is put to the ultimate test when ...
16748,C8283,C,Escape Room,Six friends test their intelligence when an es...
17026,C8561,C,Assassination Nation,After a malicious data hack exposes the secret...
12731,C4260,C,Il sogno di Calvin,A 14-year-old orphan becomes an NBA superstar ...
